In [45]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [147]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

In [260]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

In [292]:
num_classes = 10
model_tf = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28*28,)),  # Adjusted for 28x28 images
    keras.layers.Dense(128, activation='relu'),     # Increased number of neurons
    keras.layers.Dense(56, activation='relu'),      # Additional hidden layer
    keras.layers.Dense(num_classes, activation='softmax')  # Output layer for 10 classes
])

model_tf.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model_tf.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 128)               100480    
                                                                 
 dense_36 (Dense)            (None, 56)                7224      
                                                                 
 dense_37 (Dense)            (None, 10)                570       
                                                                 
Total params: 108274 (422.95 KB)
Trainable params: 108274 (422.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [293]:
# Train the model
model_tf.fit(train_images_tf, train_labels, epochs=5, batch_size=32, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.2548 - accuracy: 0.9257 - val_loss: 0.1077 - val_accuracy: 0.9710
Epoch 2/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1080 - accuracy: 0.9673 - val_loss: 0.0914 - val_accuracy: 0.9735
Epoch 3/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0743 - accuracy: 0.9768 - val_loss: 0.0848 - val_accuracy: 0.9758
Epoch 4/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0567 - accuracy: 0.9830 - val_loss: 0.0884 - val_accuracy: 0.9768
Epoch 5/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0454 - accuracy: 0.9855 - val_loss: 0.0735 - val_accuracy: 0.9787
313/313 - 0s - loss: 0.0787 - accuracy: 0.9762 - 173ms/epoch - 552us/step

Test accuracy: 0.9761999845504761


## Scaling

In [44]:
import numpy as np

# Assuming test_images_tf is your flattened and normalized test dataset
scaled_test_images_tf = np.array([int(x * 1e36) for x in test_images_tf.flatten()]).reshape(test_images_tf.shape)

In [327]:
X = test_images_tf[35:36]
print (X.shape)
pred = model_tf.predict(X)
X_in = scaled_test_images_tf[35]

np.argmax(pred, axis = 1)

(1, 784)
1/1 [==============================] - 0s 8ms/step


array([2])

### Test

In [200]:
# 784: input
# 56: first layer

# weights = [[None for _ in range(56)] for _ in range(784)]
# for i in range(784):
#     for j in range(56):
#         weights[i][j] = int(model_tf.get_weights()[0][i][j]*1e36)
#         #print (i,j)
# bias = [int(b*1e72) for b in model_tf.get_weights()[1]]


In [322]:
# Scale the weights and biases
weights_ = []
biases_= []
for layer in model_tf.layers:
    print ('new layer')
    weight, bias = layer.get_weights()
    scaled_weights = weight * 1e36
    scaled_biases = bias * 1e72
    weights_.append(scaled_weights)
    biases_.append(scaled_biases)
    

new layer
new layer
new layer


In [323]:
def relu_mod(x, p):
    return x if x < p // 2 else 0

In [324]:
def DenseInt(nInputs, nOutputs, n, input, weights, bias, act = False):
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]

    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += int(input[i] * weights[i][j])
        out[j] += int(bias[j])

        remainder[j] = str(out[j] % n)
        out[j] = out[j] // n % p

    if act:
        out = [int(relu_mod(out[i], p)) for i in range(nOutputs)]

    return Input, Weights, Bias, out, remainder
        

In [328]:
from tensorflow import keras

# Assuming model_tf is your original model
# Extract the output of the second-to-last layer (the last dense layer before softmax)
layer_output = model_tf.layers[0].output

# Create a new model that gives the output of the last dense layer
intermediate_model = keras.Model(inputs=model_tf.input, outputs=layer_output)

intermediate_model.predict(X.reshape(1, -1))  # Reshape if necessary

# 'output' now contains the output of the last dense layer before softmax

1/1 [==============================] - 0s 18ms/step


array([[3.375382  , 0.49137157, 0.        , 0.        , 0.        ,
        2.7461276 , 0.        , 1.3943845 , 0.        , 0.        ,
        0.27399603, 1.8222868 , 0.18728077, 0.        , 0.1530143 ,
        0.        , 3.81958   , 0.        , 0.        , 2.005937  ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.0538322 , 0.        , 0.        ,
        0.        , 2.4923813 , 0.        , 3.947798  , 0.        ,
        4.751608  , 1.5282661 , 3.248456  , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.114225  , 0.        ,
        0.        , 0.        , 0.77713984, 0.        , 0.        ,
        0.        , 1.5331594 , 0.        , 1.0734144 , 0.        ,
        0.        , 0.        , 3.0525808 , 0.        , 3.0161211 ,
        0.        , 1.844801  , 0.        , 0.52690274, 0.        ,
        1.366203  , 0.        , 0.29555848, 0.25

In [329]:
_, _, _, dense_out_1, remainder = DenseInt(784, 128, 10**36, X_in, weights_[0], biases_[0], act=True)
dense_out_1

[3375381873201985182738022848202033381,
 491371555592277808098225530817208883,
 0,
 0,
 0,
 2746127526227803897179978336235470710,
 0,
 1394384400855458558398448095356236748,
 0,
 0,
 273996002975310980172948651308810750,
 1822286767713078156796429369278216340,
 187280799281492287654653559384707185,
 0,
 153014248503443167407922129459733718,
 0,
 3819580052070898577451593029051574115,
 0,
 0,
 2005936983366423942113982039584815054,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1053832040409885290566714602019741974,
 0,
 0,
 0,
 2492381362501049651179951818598837780,
 0,
 3947797906256473202135487429671326501,
 0,
 4751607755044985092345537469015455317,
 1528266096741350072767256649772834509,
 3248455904660749750213052330145580302,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1114224854925594976220068868130715693,
 0,
 0,
 0,
 777139742241511025851038690606591247,
 0,
 0,
 0,
 1533159159239233791519814402303068134,
 0,
 1073414412194954880877307747578461577,
 0,
 0,
 0,
 305258061694118204565228275619423364

In [330]:
# Assuming model_tf is your original model
# Extract the output of the second-to-last layer (the last dense layer before softmax)
layer_output = model_tf.layers[1].output

# Create a new model that gives the output of the last dense layer
intermediate_model = keras.Model(inputs=model_tf.input, outputs=layer_output)

intermediate_model.predict(X.reshape(1, -1)) 

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 20ms/step


array([[1.7998114e+00, 3.5995545e+00, 1.0065154e+00, 1.1879711e+00,
        0.0000000e+00, 0.0000000e+00, 3.7056834e-01, 7.3919344e+00,
        5.1079183e+00, 4.2184820e+00, 6.3148885e+00, 0.0000000e+00,
        3.1377190e-01, 0.0000000e+00, 5.2447896e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 2.1777198e+00, 6.4400892e+00,
        0.0000000e+00, 1.7147572e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 4.8927170e-01, 5.3289118e+00, 0.0000000e+00,
        1.4893204e-01, 1.7732893e+00, 0.0000000e+00, 0.0000000e+00,
        1.7414149e+00, 0.0000000e+00, 3.9342020e+00, 0.0000000e+00,
        3.3948030e+00, 0.0000000e+00, 9.8584837e-01, 0.0000000e+00,
        2.9554210e+00, 3.7334976e+00, 2.0726490e+00, 0.0000000e+00,
        2.9684455e+00, 1.5995535e+00, 4.5307167e-03, 0.0000000e+00,
        0.0000000e+00, 1.8244205e+00, 5.4183393e+00, 1.4669970e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 2.1465123e+00]],
      dtype=float32)

In [331]:
_, _, _, dense_out_2, remainder = DenseInt(128, 56, 10**36, dense_out_1, weights_[1], biases_[1], act=True)
dense_out_2

[1799811168945183391133649237475388384,
 3599554304825136730115198564114784573,
 1006515242317991796096482474376390670,
 1187970724993925820933614262698385501,
 0,
 0,
 370568271791621067211333227519760671,
 7391933162643583765253476260882512862,
 5107917561764041572772999759861393723,
 4218480951854453722475344016106765901,
 6314887591317372780477050983263928089,
 0,
 313771887277411738014051918201356106,
 0,
 5244789399721479966223305066478368041,
 0,
 0,
 0,
 2177719625521800653722373668660703582,
 6440087986697802688884084173845061739,
 0,
 1714757121351988555720966217734536444,
 0,
 0,
 0,
 489271733204142408972072936366789643,
 5328911268936352025078491450399479805,
 0,
 148931628646414118502103920634336159,
 1773289136588075492350491395702313299,
 0,
 0,
 1741414891746437963859783522430598821,
 0,
 3934201519614926224046119429234402797,
 0,
 3394802909549501723115129918067359101,
 0,
 985848316349012788372846618805729953,
 0,
 2955420608492172885712415240469815566,
 373349731334

In [332]:
_, _, _, out_, remainder_ = DenseInt(56, 10, 10**36, dense_out_2, weights_[2], biases_[2], act=True)
out_

[0,
 5847633043705220765213876701035121626,
 12790728714326361487264012783993893283,
 319710414755301294575150273149469515,
 0,
 0,
 0,
 269239701297592828236715006057703100,
 0,
 0]

In [333]:
def mod_exp(x, p):
    return pow(int(x), 1, p)  # Modular exponentiation

def mod_softmax(out, p):
    # Apply modular exponentiation and then normalize
    exp_out = [mod_exp(x, p) for x in out]
    sum_exp_out = sum(exp_out) % p
    softmax_out = [x * pow(sum_exp_out, -1, p) % p for x in exp_out]  # Modular division
    return [int(x) for x in softmax_out]

In [334]:
model_tf.predict(X.reshape(1, -1))  # Reshape if necessary

# 'output' now contains the output of the last dense layer before softmax

1/1 [==============================] - 0s 12ms/step


array([[5.7557297e-09, 9.6433697e-04, 9.9902785e-01, 3.8324997e-06,
        4.0801211e-14, 5.9987293e-08, 8.9261287e-10, 3.6438721e-06,
        3.5281343e-07, 2.5885273e-13]], dtype=float32)

In [335]:
np.argmax(out_)

2

In [10]:
in_json = {
    "in": X_in,
    "weights": weights,
    "bias": bias,
    "out": out,
    "remainder": remainder
}

In [11]:
import json

In [12]:
with open("dense_input.json", "w") as f:
    json.dump(in_json, f)